<a href="https://colab.research.google.com/github/rodmorx/DSP_DT_WS24/blob/main/DSProjekt_DT_WS24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Sience Projekt WS24 - Dream Team

In [2]:
pip install news-please

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
pip install newspaper_scraper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 21.2 MB/s eta 0:00:00


In [28]:
!python -m spacy download de_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 24.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
from newsplease import NewsPlease
import pandas as pd
import urllib.parse
from urllib.parse import urlparse, urljoin
import newspaper_scraper as nps
import requests
from bs4 import BeautifulSoup
import csv
import sqlite3
import spacy
from spacy import displacy

### Functions to deal with URL Scraping

In [9]:
# Function to extract the domain (i.e., the newspaper/source) from the URL
def extract_newspaper(url):
    parsed_url = urllib.parse.urlparse(url)
    domain = parsed_url.netloc
    return domain.replace('www.', '')  # Clean up 'www.' if present

# Function to scrape an article from a URL and return relevant information
def scrape_article(url):
    try:
        article = NewsPlease.from_url(url)
        return {
            'title': article.title,
            'authors': article.authors,
            'date_publish': article.date_publish,
            'maintext': article.maintext,
            'url': url,
            'newspaper': extract_newspaper(url)  # Add newspaper information
        }
    except Exception as e:
        # Return None if there's any error (e.g., 404, timeout)
        print(f"Error scraping {url}: {e}")
        return None

#Function to scrape multicle articles and return a dataframe with relevant information
def scrape_articles(url_list):
    # Initialize an empty list to store scraped article data
    articles_data = []

    # Loop through each URL and scrape the article
    for url in url_list:
        article_data = scrape_article(url)
        if article_data:
            articles_data.append(article_data)

    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(articles_data)

    return df

# Mass scrape articles

First, we need to define the different functions that are needed to scrape the different websites(for example, **Bild** will need the function **nps.deBild** with its own sqlite file to work, as the other newspapers do). We will need only the URLs from this scraping

In [6]:
newspapers= [{"function": nps.DeBild(db_file='articles_bild.db'), "file": "articles_bild.de", "name":"Bild"}, {"function": nps.DeWelt(db_file='articles_welt.db'),"file": "articles_welt.de", "name": "welt"}]

[241123 17:55:03] I:49:__init__ - Initializing de_bild scraper.
I:newspaper-scraper:Initializing de_bild scraper.
[241123 17:55:03] I:49:__init__ - Initializing de_welt scraper.
I:newspaper-scraper:Initializing de_welt scraper.


Then we will iterate the scraping process through every newspaper inserted into the variable *newspapers*

In [7]:
def scrape_all_newspapers():
    for newspaper in newspapers:
        with newspaper["function"] as news:
            news.index_articles_by_date_range('2023-01-01', '2023-01-02') # CHANGE THE DATE HERE!!
            news.scrape_public_articles()

**IMPORTANT**: the db file is not saved permanently, this means that it's not possible to do queries once the database is done, and that's a problem of the original library. To resolve it, you need to go to the *newspaper_please* 's *database.py* file, and paste this at **line 219**, inside the try/catch nest, instead of the line already written inside:




```
with sqlite3.connect(self.db_file) as conn:
  _df.to_sql(table_name, conn, if_exists=mode, index_label='URL')
  conn.commit()
```



With this out of the mind, now we can connect and do queries to the database:

In [29]:
nlp = spacy.load("de_core_news_md")

In [25]:
def scrape_and_analyze(name):
  cnx = sqlite3.connect(name) # Name is the file path to the database

  urls = pd.read_sql_query("SELECT * FROM {}".format("tblArticlesIndexed"), cnx) # We do a query that returns a dataframe from the table tblArticlesIndexed

  df = scrape_articles(urls['URL'].head())
  docs = nlp.pipe(df['maintext'])
  displacy.render(docs,style='ent', jupyter=True)

In [ ]:
#  docs = nlp.pipe(df['maintext'])
#  displacy.render(docs,style='ent', jupyter=True)

Example of the use of the two functions:

In [30]:
scrape_all_newspapers()
scrape_and_analyze("articles_welt.db")

[241123 18:27:33] I:130:_load_table - Load database from articles_bild.db.
I:newspaper-scraper:Load database from articles_bild.db.
[241123 18:27:33] I:158:_load_table - Loaded 471 articles from database. 89.8% scraped and 0.0% processed.
I:newspaper-scraper:Loaded 471 articles from database. 89.8% scraped and 0.0% processed.
[241123 18:27:33] I:182:index_articles_by_date_range - No new days to scrape. Pass skip_existing=False to scrape all days again.
I:newspaper-scraper:No new days to scrape. Pass skip_existing=False to scrape all days again.
[241123 18:27:33] I:328:scrape_public_articles - No articles to scrape.
I:newspaper-scraper:No articles to scrape.
[241123 18:27:33] I:130:_load_table - Load database from articles_welt.db.
I:newspaper-scraper:Load database from articles_welt.db.
[241123 18:27:33] I:158:_load_table - Loaded 125 articles from database. 0.0% scraped and 0.0% processed.
I:newspaper-scraper:Loaded 125 articles from database. 0.0% scraped and 0.0% processed.
[241123 